# Imports & Dados

## Imports & Configs

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')

import inflection

## Carregamento dos Dados

In [2]:
file_path = 'acidentes_brasil - acidentes_brasil.csv'

df_raw = pd.read_csv(file_path)

df_raw.sample(3)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
4354,280119,16/04/2020,quinta-feira,20:00:00,SP,153,41.9,ONDA VERDE,Desobediencia as normas de transito pelo condutor,Colisao frontal,Com Vitimas Feridas,Plena Noite,Crescente,Ceu Claro,Simples,Reta,Nao,3,0,0,1,2,0,1,2,"-20,63748646","-49,33845933",SPRF-SP,DEL09-SP,UOP01-DEL09-SP
255631,565340,04/12/2023,segunda-feira,06:00:00,BA,116,776.2,PLANALTO,Transitar no Acostamento,Derramamento de carga,Com Vitimas Feridas,Pleno dia,Decrescente,Ceu Claro,Simples,Reta,Nao,2,0,1,0,1,0,1,2,"-14,59897899","-40,43763203",SPRF-BA,DEL08-BA,UOP02-DEL08-BA
101105,364191,14/06/2021,segunda-feira,15:30:00,RN,101,171.0,CANGUARETAMA,Condutor Dormindo,Saida de leito carrocavel,Com Vitimas Feridas,Pleno dia,Decrescente,Sol,Dupla,Reta,Nao,4,0,4,0,0,0,4,1,"-6,45751198","-35,11047927",SPRF-RN,DEL01-RN,UOP01-DEL01-RN


## Funções

In [3]:
def estacao_do_ano(data):
    if pd.isna(data):
        return np.nan
    
    dia = data.day
    mes = data.month

    # Verão
    if (mes == 12 and dia >= 21) or mes in [1, 2] or (mes == 3 and dia < 21):
        return "Verão"
    # Outono
    elif (mes == 3 and dia >= 21) or mes in [4, 5] or (mes == 6 and dia < 21):
        return "Outono"
    # Inverno
    elif (mes == 6 and dia >= 21) or mes in [7, 8] or (mes == 9 and dia < 23):
        return "Inverno"
    # Primavera
    else:
        return "Primavera"
    
    
    
    
    
def periodo_do_dia(horario):
    if pd.isna(horario):
        return np.nan
    hora = horario.hour
    if 0 <= hora < 6:
        return "Madrugada"
    elif 6 <= hora < 12:
        return "Manhã"
    elif 12 <= hora < 18:
        return "Tarde"
    else:
        return "Noite"

# Descrição dos Dados

In [4]:
df1 = df_raw.copy()

## Normalização colunas

In [5]:
original_cols = df1.columns.tolist()

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, original_cols))

df1.columns = cols_new
print(cols_new)

['id', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos', 'ignorados', 'feridos', 'veiculos', 'latitude', 'longitude', 'regional', 'delegacia', 'uop']


## Dimensão dos Dados

In [6]:
print(f'Number of Rows: {df1.shape[0]}')
print(f'Number of Columns: {df1.shape[1]}')

Number of Rows: 260524
Number of Columns: 30


## Data Types

In [7]:
df1["data_inversa"] = (
    df1["data_inversa"]
    .astype(str)
    .str.strip()
)

df1["data_inversa"] = pd.to_datetime(
    df1["data_inversa"],
    errors="coerce",
    dayfirst=True
)

df1["horario"] = (
    df1["horario"]
    .astype(str)
    .str.strip()
)

df1["horario"] = pd.to_datetime(
    df1["horario"],
    format="%H:%M:%S",
    errors="coerce"
).dt.time

for col in ["latitude", "longitude"]:
    df1[col] = (
        df1[col]
        .astype(str)
        .str.strip()
        .str.replace(",", ".", regex=False)
    )
    df1[col] = pd.to_numeric(df1[col], errors="coerce")



print(df1.dtypes)
df1.sample(3)

id                                object
data_inversa              datetime64[ns]
dia_semana                        object
horario                           object
uf                                object
br                                 int64
km                               float64
municipio                         object
causa_acidente                    object
tipo_acidente                     object
classificacao_acidente            object
fase_dia                          object
sentido_via                       object
condicao_metereologica            object
tipo_pista                        object
tracado_via                       object
uso_solo                          object
pessoas                            int64
mortos                             int64
feridos_leves                      int64
feridos_graves                     int64
ilesos                             int64
ignorados                          int64
feridos                            int64
veiculos        

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
169558,461967,2022-07-16,sabado,07:50:00,SP,116,104.5,TAUBATE,Reacao tardia ou ineficiente do condutor,Engavetamento,Com Vitimas Feridas,Pleno dia,Decrescente,Ceu Claro,Dupla,Reta,Nao,5,0,3,0,2,0,3,3,-23.003402,-45.506269,SPRF-SP,DEL08-SP,UOP01-DEL08-SP
41493,298143,2020-07-29,quarta-feira,07:00:00,BA,235,251.3,JUAZEIRO,Falta de Atencao a Conducao,Colisao frontal,Com Vitimas Feridas,Pleno dia,Decrescente,Nublado,Simples,Reta,Sim,2,0,0,1,0,1,1,2,-9.430489,-40.503227,SPRF-BA,DEL07-BA,UOP01-DEL07-BA
87306,341374,2021-02-18,quinta-feira,08:30:00,ES,262,6.6,CARIACICA,Condutor desrespeitou a iluminacao vermelha do...,Colisao transversal,Com Vitimas Feridas,Pleno dia,Decrescente,Ceu Claro,Multipla,Reta,Sim,3,0,0,1,1,1,1,3,-20.340075,-40.394513,SPRF-ES,DEL01-ES,UOP01-DEL01-ES


## Checando NA

In [8]:
df1.isna().sum()

id                          0
data_inversa                0
dia_semana                  0
horario                     0
uf                          0
br                          0
km                          0
municipio                   0
causa_acidente              0
tipo_acidente               1
classificacao_acidente      5
fase_dia                    0
sentido_via                 0
condicao_metereologica      0
tipo_pista                  0
tracado_via                 0
uso_solo                    0
pessoas                     0
mortos                      0
feridos_leves               0
feridos_graves              0
ilesos                      0
ignorados                   0
feridos                     0
veiculos                    0
latitude                    0
longitude                   0
regional                    7
delegacia                  78
uop                       169
dtype: int64

In [9]:
df1 = df1.dropna()
df1.isna().sum()

id                        0
data_inversa              0
dia_semana                0
horario                   0
uf                        0
br                        0
km                        0
municipio                 0
causa_acidente            0
tipo_acidente             0
classificacao_acidente    0
fase_dia                  0
sentido_via               0
condicao_metereologica    0
tipo_pista                0
tracado_via               0
uso_solo                  0
pessoas                   0
mortos                    0
feridos_leves             0
feridos_graves            0
ilesos                    0
ignorados                 0
feridos                   0
veiculos                  0
latitude                  0
longitude                 0
regional                  0
delegacia                 0
uop                       0
dtype: int64

# Feature Engineering

In [10]:
df2 = df1.copy()

## Hipóteses

### Análise Temporal

### Análise Geográfica

### Condicionantes do Acidente

### Comparações Regionais e Temporais

### Identificação de Trechos Críticos

## Feature Engineering

In [11]:
df2['ano'] = df2['data_inversa'].dt.year

df2['mes'] = df2['data_inversa'].dt.month

df2['dia'] = df2['data_inversa'].dt.day

df2["fim_de_semana"] = df2["dia_semana"].isin(["domingo", "sabado"])

df2["estacao"] = df2["data_inversa"].apply(estacao_do_ano)

In [12]:
df2.to_csv( 'dados limpos.csv', index=False )

# Analise Exploratoria de Dados